In [2]:
import numpy as np
import sympy as sym

In [55]:
# define variables
# altitude alpha and azimuth phi for the polar coordinate
a = sym.Symbol('alpha')
phi = sym.Symbol('phi')
# m and n in Krivenek notation
n = sym.Symbol('m')
m = sym.Symbol('n')
# a0 for the aperture limit
a0 = sym.Symbol('alpha_0')

In [194]:
# calculate aberration function chi, up to n = 5 
chi = 0
# define the maximum order to be considered in the aberration function
order = 3
# define len = 5 list to save all the Cnm
C = [None] * order
for idx_n in range(order):
    n = idx_n + 1
    # find all possible m for a given n
    m_list = []
    i = n + 1
    while i >= 0:
        m_list.append(i)
        i -= 2
    # reverse the order of m_list so that it start with small number
    m_list = m_list[::-1]
    Cn = [None] * len(m_list)
    for idx_m in range(len(m_list)):
        Cnm_str = 'C_' + str(n) + str(m_list[idx_m])
        print(Cnm_str)
        Cn[idx_m] = sym.Symbol(Cnm_str)
        chi += sym.cos(m_list[idx_m]*phi)/(n+1)*a**(n+1)*Cn[idx_m]
    C[idx_n] = Cn
chi

C_10
C_12
C_21
C_23
C_30
C_32
C_34


C_10*alpha**2/2 + C_12*alpha**2*cos(2*phi)/2 + C_21*alpha**3*cos(phi)/3 + C_23*alpha**3*cos(3*phi)/3 + C_30*alpha**4/4 + C_32*alpha**4*cos(2*phi)/4 + C_34*alpha**4*cos(4*phi)/4

In [195]:
# vector for gradient of chi
g = [sym.diff(chi,a), sym.diff(chi,phi)]
# vector for coordinate in k space
ang = [a, phi]

In [196]:
g[0]

C_10*alpha + C_12*alpha*cos(2*phi) + C_21*alpha**2*cos(phi) + C_23*alpha**2*cos(3*phi) + C_30*alpha**3 + C_32*alpha**3*cos(2*phi) + C_34*alpha**3*cos(4*phi)

In [197]:
g[1]

-C_12*alpha**2*sin(2*phi) - C_21*alpha**3*sin(phi)/3 - C_23*alpha**3*sin(3*phi) - C_32*alpha**4*sin(2*phi)/2 - C_34*alpha**4*sin(4*phi)

In [198]:
ang[0]

alpha

In [199]:
ang[1]

phi

In [221]:
# calculate three integrations for emittance
emit1 = (g[0]*g[0] + g[1]*g[1])/(sym.pi * a0 * a0)
emit1 = sym.integrate(emit1, (phi, 0, sym.pi * 2))
emit1 = sym.integrate(emit1, (a, 0, a0))
emit1

alpha_0**7*(C_32**2 + 4*C_34**2)/36 + alpha_0**5*(9*C_12*C_32 + C_21**2 + 9*C_23**2 + 18*C_30**2 + 9*C_32**2 + 9*C_34**2)/63 + alpha_0**3*(4*C_10*C_30 + C_12**2 + 2*C_12*C_32 + C_21**2 + C_23**2)/5 + alpha_0*(2*C_10**2 + C_12**2)/3

In [222]:
emit2 = (ang[0]*ang[0] + ang[1]*ang[1])/(sym.pi * a0 * a0)
emit2 = sym.integrate(emit2, (phi, 0, sym.pi * 2))
emit2 = sym.integrate(emit2, (a, 0, a0))
emit2

2*alpha_0/3 + 8*pi**2/(3*alpha_0)

In [223]:
emit3 = (ang[0]*g[0] + ang[1]*g[1])/(sym.pi * a0 * a0)
emit3 = sym.integrate(emit3, (phi, 0, sym.pi * 2))
emit3 = sym.integrate(emit3, (a, 0, a0))
emit3

alpha_0**3*(4*C_30 + C_32 + C_34)/10 + alpha_0**2*(C_21 + C_23)/6 + alpha_0*(2*C_10 + C_12)/3

In [224]:
emit = emit1*emit2 - emit3*emit3
sym.expand(emit)

16*pi**2*C_10**2/9 - 4*C_10*C_12*alpha_0**2/9 - 2*C_10*C_21*alpha_0**3/9 - 2*C_10*C_23*alpha_0**3/9 + 32*pi**2*C_10*C_30*alpha_0**2/15 - 2*C_10*C_32*alpha_0**4/15 - 2*C_10*C_34*alpha_0**4/15 + 2*C_12**2*alpha_0**4/15 + C_12**2*alpha_0**2/9 + 8*pi**2*C_12**2*alpha_0**2/15 + 8*pi**2*C_12**2/9 - C_12*C_21*alpha_0**3/9 - C_12*C_23*alpha_0**3/9 - 4*C_12*C_30*alpha_0**4/15 + 2*C_12*C_32*alpha_0**6/21 + C_12*C_32*alpha_0**4/5 + 8*pi**2*C_12*C_32*alpha_0**4/21 + 16*pi**2*C_12*C_32*alpha_0**2/15 - C_12*C_34*alpha_0**4/15 + 2*C_21**2*alpha_0**6/189 + 19*C_21**2*alpha_0**4/180 + 8*pi**2*C_21**2*alpha_0**4/189 + 8*pi**2*C_21**2*alpha_0**2/15 - C_21*C_23*alpha_0**4/18 - 2*C_21*C_30*alpha_0**5/15 - C_21*C_32*alpha_0**5/30 - C_21*C_34*alpha_0**5/30 + 2*C_23**2*alpha_0**6/21 + 19*C_23**2*alpha_0**4/180 + 8*pi**2*C_23**2*alpha_0**4/21 + 8*pi**2*C_23**2*alpha_0**2/15 - 2*C_23*C_30*alpha_0**5/15 - C_23*C_32*alpha_0**5/30 - C_23*C_34*alpha_0**5/30 + 16*C_30**2*alpha_0**6/525 + 16*pi**2*C_30**2*alpha_0**4/

In [229]:
sym.integrate(sym.cos(3*x) * sym.cos(3*x), (x, 0, sym.pi * 2))

pi

In [226]:
C[0][0]

C_10

In [227]:
sym.diff(emit,C[0][0])

-4*alpha_0*(alpha_0**3*(4*C_30 + C_32 + C_34)/10 + alpha_0**2*(C_21 + C_23)/6 + alpha_0*(2*C_10 + C_12)/3)/3 + (2*alpha_0/3 + 8*pi**2/(3*alpha_0))*(4*C_10*alpha_0/3 + 4*C_30*alpha_0**3/5)